<img src="images/one_logo.png" width="500"/> 

# Polytope example notebook - Moldel level data from High Resolution model

This notebook will show you basic use of Polytope client.  
It should be used as skeleton example for building Polytope API requests.

First we import Polytope Client and xarray library to inspect the data.

In [ ]:
from polytope.api import Client
import xarray as xr

In order to use the Client, we need to start it.  
We can then list available collections. In HiDALGO Project users have the option to use 'hidalgo-mars' collection.

Insert here your email and ecmwf token, or username and password:

In [ ]:
c = Client(username = 'test1', password = 'password1')

In [ ]:
c = Client(user_email = 'johndoe@ecmwf.int',
           user_key = '4j3s3d34n4sn335jacf3n3d4f4g61635')

In [ ]:
c.list_collections()

### Get model the temperature data on model levels
This example will get us data on all 137 model levels, from 00 model run, first 12 steps from each run (0 to 11), for area around Hungary and at 0.5 by 0.5 degree resolution.  

To build efficient Polytope request for data and spend least time in the queue, it is recomended that each request access only one tape with data.   
For the data on Model Levels data from one forecast run (one date and one time) are on one tape. This means that it is recomended to have a separate request for each time and each date.

Model Level data is direct model output, not interpolated into any other levels. Latest version of IFS model has 137 levels, with 0 being the highest and 137 being the lowest level.
You can read more about IFS model leves on [this link](https://www.ecmwf.int/en/forecasts/documentation-and-support/137-model-levels).

In order to calculate parameter values on height levels you need to download all 137 model levels. 

When you are trying out requests, try coarser resolution until you're happy, so you get smaller files, and when you're happy, increase it to 0.1/0.1.

In [ ]:
request_00 = {
    'stream': 'oper',
    'levtype': 'ml',
    'levelist': '1/to/137',
    'param': 't',
    'step': '0/6/12',
    'time': '00',
    'date': '20170305',
    'type': 'fc',
    'class': 'od',
    'expver': '0001',
    'area': '50/15/40/25',
    'grid': '0.5/0.5'
}

In [ ]:
c.retrieve('hidalgo-mars', request_00, 'data/ml_t.grib')

### Submitting a list of requests
Sometimes we want to split our request into 2 smaller requests. We can then submit both requests together as a list.  
Let's make a request that accesses the same set of data from the same date, just run from 12 UTC.

In [ ]:
request_12 = {
    'stream': 'oper',
    'levtype': 'ml',
    'levelist': '1/to/137',
    'param': 't',
    'step': '0/6/12',
    'time': '12',
    'date': '20170305',
    'type': 'fc',
    'class': 'od',
    'expver': '0001',
    'area': '50/15/40/25',
    'grid': '0.5/0.5'
}

These requests can then be submitted together. The result will be downloaded in the same file.

In [ ]:
c.retrieve('hidalgo-mars', [request_00,request_12], 'data/ml_t.grib')

### Asynchronous retrieval 

If we want to submit a request but run it in background so we can do other things, asynchronous retrieval can also be done. 

In [ ]:
r = c.retrieve('hidalgo-mars', [request_00,request_12], 'data/ml_t.grib',
               asynchronous = True)

Now we can see the info of our requests or download the data when it is ready.  
Each request is a member of the list so we can access each of them individually (`r[0]` or `r[1]` in our case).

In [ ]:
r[1].describe()

And download.  
You can only dowlnoad each request from the list separately. By default they will be appended in the same file.  If for any reason you don't want this you can set parameter append=Flase.

In [ ]:
r[0].download()

In [ ]:
r[1].download()

We can inspect what is in our grib using xarray. Note that, when loaded in xarray vertical levels are called **hybrid**

In [ ]:
ds = xr.open_dataset('data/ml_t.grib',engine='cfgrib')
ds

In [ ]:
ds.t.sel(time='2017-03-05 12:00:00',step=('05:00:00'),hybrid=3).plot()